This code shows the demonstration of bag of words. (It is just for requirement)

# EDA

EDA processes is almost same as the main program

Importing main libraries

In [ ]:
import numpy as np 
import pandas as pd
import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))
import tensorflow as tf

In [ ]:
dataframe = pd.read_csv('/kaggle/input/iba-ml1-final-project/train.csv')
dataframe.drop(["Id"], axis = 1, inplace = True)
dataframe

In [ ]:
one_hot_Rating = pd.get_dummies(dataframe.Rating,prefix="Rating")
dataframe = dataframe.drop('Rating',axis = 1)
dataframe = dataframe.join(one_hot_Rating)
dataframe

In [ ]:
dataframe.drop(dataframe[dataframe.Review_Title.isnull() & dataframe.Review.isnull()].index, inplace = True)
dataframe = dataframe.reset_index().drop(['index'],axis = 1)
dataframe

In [ ]:
from nltk.corpus import wordnet

def get_pos(word):
    tag = nltk.pos_tag([word])[0][1][0]
    tag_dict = {"J": wordnet.ADJ,
                "N": wordnet.NOUN,
                "V": wordnet.VERB,
                "R": wordnet.ADV}
    element = tag_dict.get(tag)
    if(element == "J" or element=="N" or element=="V" or element=="R"):
        return element
    return 'n' #it is just like for not getting the error, we pass 'n' if the pos tag couldn't found or is not in the dictionary.

In [ ]:
import re
import nltk
from nltk.corpus import stopwords
from nltk.stem.porter import PorterStemmer
from nltk.stem import WordNetLemmatizer


dataframe['Review_Title'] = dataframe['Review_Title'].fillna('None')
dataframe['Review'] = dataframe['Review'].fillna('None')
pd.set_option('mode.chained_assignment', None)
dataframe.Review = dataframe.Review.str.lower()
dataframe.Review_Title = dataframe.Review_Title.str.lower()
for i in range(0,len(dataframe)):
    review = re.sub('[^a-zA-z]', ' ', dataframe['Review'][i])
    review = review.split()
    wl = WordNetLemmatizer()
    all_stopwords = stopwords.words('english')
    all_stopwords.remove('not')    
    review = [wl.lemmatize(word, pos = get_pos(word)) for word in review if word not in set(all_stopwords)]
    review = ' '.join(review)
    dataframe['Review'][i] = review
    
    review_title = re.sub('[^a-zA-z]', ' ', dataframe['Review_Title'][i])
    review_title = review_title.split()
    review_title = [wl.lemmatize(word, pos = get_pos(word)) for word in review_title if word not in set(all_stopwords)]
    review_title = ' '.join(review_title)
    dataframe['Review_Title'][i] = review_title

In [ ]:
dataframe[['Review_Title','Review']]

In [ ]:
from sklearn.model_selection import train_test_split
X_dataframe = dataframe.iloc[:,[1,2]]
y_dataframe = dataframe.iloc[:,-6:]

X_train, X_test, y_train, y_test = train_test_split(X_dataframe,y_dataframe, stratify = y_dataframe.Recommended)

In [ ]:
X_train_review_title_input = X_train[['Review_Title']].squeeze()
X_train_review_input = X_train[['Review']].squeeze()

X_test_review_title_input = X_test[['Review_Title']].squeeze()
X_test_review_input = X_test[['Review']].squeeze()

y_train_rating_output = y_train[['Rating_1','Rating_2','Rating_3','Rating_4','Rating_5']]
y_train_recommended_output = y_train[['Recommended']]


y_test_rating_output = y_test[['Rating_1','Rating_2','Rating_3','Rating_4','Rating_5']]
y_test_recommended_output = y_test[['Recommended']]

In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer
tf_converter_review = TfidfVectorizer(max_features=10000)
tf_converter_review_title = TfidfVectorizer(max_features=10000)

tf_review_title = tf_converter_review_title.fit_transform(X_train_review_title_input).toarray()
tf_review = tf_converter_review.fit_transform(X_train_review_input).toarray()

tf_review_title_test = tf_converter_review_title.transform(X_test_review_title_input).toarray()
tf_review_test = tf_converter_review.transform(X_test_review_input).toarray()

In [ ]:
tf_review_title_test.shape

# Model

Now let's create our model. As I said, we will use function api for it.

In [ ]:
from tensorflow import keras
from tensorflow.keras import layers

input_review_title = keras.Input(shape=(tf_review_title.shape[1],), name="review_title")
input_review = keras.Input(shape=(tf_review.shape[1],), name="review")

dense_review_title = layers.Dense(4,name = 'dense_review_title')(input_review_title)
dense_review = layers.Dense(8,name = 'dense_review')(input_review)

concat_layer = layers.concatenate([dense_review_title,dense_review])

rating_pred = layers.Dense(5,activation ='softmax',name="rating")(concat_layer)
recommended_pred = layers.Dense(1,activation ='sigmoid',name="recommended")(concat_layer)

model = keras.Model(
    inputs=[input_review_title, input_review],
    outputs=[rating_pred, recommended_pred],
)

In [ ]:
keras.utils.plot_model(model,show_shapes=True)

In [ ]:
model.compile(optimizer='adam', loss = {"rating": "categorical_crossentropy",
                                        "recommended": "binary_crossentropy"},
                                metrics={'rating': 'accuracy', 
                                         'recommended': 'accuracy'})

In [ ]:
callback_loss = tf.keras.callbacks.EarlyStopping(monitor='loss',patience = 5)
callback_accuracy = tf.keras.callbacks.EarlyStopping(monitor='val_rating_accuracy', patience = 3)
history = model.fit({"review_title": tf_review_title,
                     "review": tf_review}, 
                    {"rating": y_train_rating_output,
                     "recommended":y_train_recommended_output}, 
                    epochs=50,batch_size=4, verbose=1, 
                    validation_data = ({"review_title": tf_review_title_test,
                                        "review": tf_review_test},
                                         {"rating": y_test_rating_output,
                                           "recommended":y_test_recommended_output}),
                   callbacks=[callback_loss, callback_accuracy], shuffle = True)

# Interpretation

In [ ]:
import matplotlib.pyplot as plt
print(history.history.keys())

fig, axs = plt.subplots(2, 2, figsize = (20,10))
axs[0, 0].plot(history.history['rating_loss'])
axs[0, 0].set_title('rating_loss')
axs[0, 1].plot(history.history['recommended_loss'], 'tab:orange')
axs[0, 1].set_title('recommended_loss]')
axs[1, 0].plot(history.history['val_rating_loss'], 'tab:green')
axs[1, 0].set_title('val_rating_loss')
axs[1, 1].plot(history.history['val_recommended_loss'], 'tab:red')
axs[1, 1].set_title('val_recommended_loss')

As you can see the bag of words approach is too bad for this task

## Applying to the test set


In [ ]:
test_dataframe = pd.read_csv('/kaggle/input/iba-ml1-final-project/test.csv')
test_dataframe_Id = test_dataframe["Id"]
test_dataframe.drop(["Id"], axis = 1, inplace = True)
test_dataframe

Applying the same NLP techniques



In [ ]:
test_dataframe['Review_Title'] = test_dataframe['Review_Title'].fillna('None')
test_dataframe['Review'] = test_dataframe['Review'].fillna('None')
#Now
test_dataframe.Review = test_dataframe.Review.str.lower()
test_dataframe.Review_Title = test_dataframe.Review_Title.str.lower()
for i in range(0,len(test_dataframe)):
    review = re.sub('[^a-zA-z]', ' ', test_dataframe['Review'][i])
    review = review.split()
    wl = WordNetLemmatizer()
    all_stopwords = stopwords.words('english')
    all_stopwords.remove('not')    
    review = [wl.lemmatize(word, pos = get_pos(word)) for word in review if word not in set(all_stopwords)]
    review = ' '.join(review)
    test_dataframe['Review'][i] = review
    
    review_title = re.sub('[^a-zA-z]', ' ', test_dataframe['Review_Title'][i])
    review_title = review_title.split()
    review_title = [wl.lemmatize(word, pos = get_pos(word)) for word in review_title if word not in set(all_stopwords)]
    review_title = ' '.join(review_title)
    test_dataframe['Review_Title'][i] = review_title

Seperate columns for input

In [ ]:
test_df_review_title_input = tf_converter_review_title.transform(test_dataframe[['Review_Title']].squeeze()).toarray()
test_df_review_input = tf_converter_review.transform(test_dataframe[['Review']].squeeze()).toarray()

Now let's predict our model

In [ ]:
test_pred = model.predict({"review_title": test_df_review_title_input,
                           "review": test_df_review_input})

Change the probabilities to appropriate numbers.

In [ ]:
rating_answer = np.argmax(np.array(test_pred[0]),axis=1)+1
recom_answer = (test_pred[1] > 0.5).astype(int).flatten()

Let's put them to the pandas dataframe with proper columns

In [ ]:
df_answer = pd.DataFrame({"Id":test_dataframe_Id,
                          "Rating":rating_answer,
                          "Recommended": recom_answer})
df_answer